In [25]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score



In [21]:
df = pd.read_csv('clean_tabular_data.csv')


In [23]:

def load_airbnb(csv_file_path):
    df = pd.read_csv(csv_file_path)
    features = df.drop(columns= ['ID', 'Category', 'Title', 'Description', 'Amenities', 'Location', 'url'])
    labels = df["Price_Night"]
    loaded_data = (features,labels)
    return loaded_data


In [29]:
class AirbnbNightlyPriceDataset(Dataset):
    def __init__(self):
        # Convert the features and labels dataframes into torch tensors
        super().__init__()
        self.X, self.y = load_airbnb('clean_tabular_data.csv')
        assert len(self.X) == len(self.y) # Data and labels have to be of equal length

    def __getitem__(self, index):
        return (torch.tensor(self.X[index]), torch.tensor(self.y[index]))

    def __len__(self):
        return len(self.X)


# create dataset
dataset = AirbnbNightlyPriceDataset()

# get first sample and unpack
first_data = dataset[10]
features, labels = first_data
print(features, labels)

# Load whole dataset with DataLoader
# shuffle: shuffle data, good for training
# num_workers: faster loading with multiple subprocesses
# !!! IF YOU GET AN ERROR DURING LOADING, SET num_workers TO 0 !!!
train_loader = DataLoader(dataset=dataset,
                          batch_size=4,
                          shuffle=True,
                          num_workers=2)



KeyError: 10

In [ ]:
def split_data(dataset): #TODO scalar the data
    """Splits the input dataset into training, validation, and testing sets.

    Returns:
        - train_dataset: a PyTorch Dataset object containing 70% of the input data, to be used for training.
        - validation_dataset: a PyTorch Dataset object containing 15% of the input data, to be used for validation.
        - test_dataset: a PyTorch Dataset object containing 15% of the input data, to be used for testing."""

    # Splits data into 70% training and 30% test
    train_dataset, test_dataset = random_split(dataset, [int(len(dataset) * 0.7), len(dataset)-int(len(dataset)*0.7)])

    # Splits test data in half, percentage of total dataset is 15% test and 15% validation
    validation_dataset, test_dataset = random_split(test_dataset, [int(len(test_dataset) * 0.5), len(test_dataset)-int(len(test_dataset)*0.5)])

    print(f"    Training: {len(train_dataset)}")
    print(f"    Validation: {len(validation_dataset)}")
    print(f"    Testing: {len(test_dataset)}")


    return train_dataset, validation_dataset, test_dataset
